System version: 3.7.9 (tags/v3.7.9:13c94747c7, Aug 17 2020, 18:58:18) [MSC v.1900 64 bit (AMD64)] <br>
Tensorflow version: 2.6.1

In [ ]:
!pip install retrying
!pip install yaml
!pip install nbformat
!pip install nbconvert
!pip install --upgrade typing_extensions
!pip install recommenders --no-deps

In [ ]:
import os
import sys
import numpy as np
import zipfile
from tqdm import tqdm
from tempfile import TemporaryDirectory
import tensorflow as tf
tf.get_logger().setLevel('ERROR')
import recommenders
from recommenders.models.deeprec.deeprec_utils import download_deeprec_resources 
from recommenders.models.newsrec.newsrec_utils import prepare_hparams
from recommenders.models.newsrec.models.naml import NAMLModel
from recommenders.models.newsrec.io.mind_all_iterator import MINDAllIterator
from recommenders.models.newsrec.newsrec_utils import get_mind_data_set
from recommenders.utils.notebook_utils import store_metadata

print("System version: {}".format(sys.version))
print("Tensorflow version: {}".format(tf.__version__))


System version: 3.7.9 (tags/v3.7.9:13c94747c7, Aug 17 2020, 18:58:18) [MSC v.1900 64 bit (AMD64)]
Tensorflow version: 2.6.1


## Prepare Parameters

In [ ]:
epochs = 5
seed = 42
batch_size = 32

# demo, small, large
MIND_type = 'demo'

## Download and load data

In [ ]:
model_path = "..."   # path to save the model weights
data_path = "..."    # path to the data
train_news_file = os.path.join(data_path, 'train', r'news.tsv')
train_behaviors_file = os.path.join(data_path, 'train', r'behaviors.tsv')
valid_news_file = os.path.join(data_path, 'valid', r'news.tsv')
valid_behaviors_file = os.path.join(data_path, 'valid', r'behaviors.tsv')
wordEmb_file = os.path.join(data_path, "utils", "embedding_all.npy")
userDict_file = os.path.join(data_path, "utils", "uid2index.pkl")
wordDict_file = os.path.join(data_path, "utils", "word_dict_all.pkl")
vertDict_file = os.path.join(data_path, "utils", "vert_dict.pkl")
subvertDict_file = os.path.join(data_path, "utils", "subvert_dict.pkl")
yaml_file = os.path.join(data_path, "utils", r'naml.yaml')

mind_url, mind_train_dataset, mind_dev_dataset, mind_utils = get_mind_data_set(MIND_type)

if not os.path.exists(train_news_file):
    download_deeprec_resources(mind_url, os.path.join(data_path, 'train'), mind_train_dataset)
    
if not os.path.exists(valid_news_file):
    download_deeprec_resources(mind_url, \
                               os.path.join(data_path, 'valid'), mind_dev_dataset)
if not os.path.exists(yaml_file):
    download_deeprec_resources(r'https://recodatasets.z20.web.core.windows.net/newsrec/', \
                               os.path.join(data_path, 'utils'), mind_utils)

100%|██████████| 17.0k/17.0k [00:25<00:00, 669KB/s]
100%|██████████| 9.84k/9.84k [00:14<00:00, 675KB/s]
100%|██████████| 95.0k/95.0k [02:20<00:00, 674KB/s]


In [ ]:
import os
print(os.getcwd())

## Create hyper-parameters

In [26]:
hparams = prepare_hparams(yaml_file, 
                          vert_num=max(vert_dict.values()) + 1,  
                          subvert_num=max(subvert_dict.values()) + 1,
                          batch_size=32,
                          epochs=5,
                          wordEmb_file=wordEmb_file,
                          wordDict_file=wordDict_file, 
                          userDict_file=userDict_file,
                          vertDict_file=vertDict_file, 
                          subvertDict_file=subvertDict_file,
                          )


In [27]:
iterator = MINDAllIterator

## Train the NAML model

In [28]:
model = NAMLModel(hparams, iterator, seed=seed)

In [ ]:
import pickle

with open(vertDict_file, "rb") as f:
    vert_dict = pickle.load(f)

with open(subvertDict_file, "rb") as f:
    subvert_dict = pickle.load(f)

print("Max index in vert_dict:", max(vert_dict.values()))
print("Max index in subvert_dict:", max(subvert_dict.values()))

In [29]:
%%time
model.fit(train_news_file, train_behaviors_file, valid_news_file, valid_behaviors_file)

1085it [1:02:26,  3.45s/it]
0it [00:00, ?it/s]d:\College D drive\Samsungggg PRISM\NAML\naml_env\lib\site-packages\keras\engine\training.py:2470: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '
18693it [00:52, 357.37it/s]
7507it [03:49, 32.77it/s]
7538it [00:01, 7414.87it/s]


at epoch 1
train info: logloss loss:1.4898013003960182
eval info: group_auc:0.5863, mean_mrr:0.2587, ndcg@10:0.3485, ndcg@5:0.2841
at epoch 1 , train time: 3746.5 eval time: 288.4


1085it [1:02:51,  3.48s/it]
18693it [00:50, 367.32it/s]
7507it [03:24, 36.75it/s]
7538it [00:01, 6383.59it/s]


at epoch 2
train info: logloss loss:1.4130157224593625
eval info: group_auc:0.6247, mean_mrr:0.2901, ndcg@10:0.3824, ndcg@5:0.3226
at epoch 2 , train time: 3771.3 eval time: 263.3


1085it [1:03:03,  3.49s/it]
18693it [00:49, 376.76it/s]
7507it [03:21, 37.20it/s]
7538it [00:00, 7746.97it/s]


at epoch 3
train info: logloss loss:1.3693901009273968
eval info: group_auc:0.6357, mean_mrr:0.2824, ndcg@10:0.381, ndcg@5:0.3189
at epoch 3 , train time: 3783.7 eval time: 258.3


1085it [1:02:16,  3.44s/it]
18693it [00:49, 379.99it/s]
7507it [03:26, 36.39it/s]
7538it [00:00, 7753.16it/s]


at epoch 4
train info: logloss loss:1.3387556996763028
eval info: group_auc:0.6451, mean_mrr:0.2995, ndcg@10:0.3948, ndcg@5:0.3343
at epoch 4 , train time: 3736.3 eval time: 262.4


1085it [1:03:27,  3.51s/it]
18693it [00:49, 376.05it/s]
7507it [03:25, 36.46it/s]
7538it [00:00, 7751.65it/s]


at epoch 5
train info: logloss loss:1.320486850947279
eval info: group_auc:0.6448, mean_mrr:0.2976, ndcg@10:0.3912, ndcg@5:0.326
at epoch 5 , train time: 3807.8 eval time: 262.4
Wall time: 5h 36min 20s


In [31]:
%%time
res_syn = model.run_eval(valid_news_file, valid_behaviors_file)
print(res_syn)

18693it [00:46, 398.83it/s]
7507it [03:24, 36.79it/s]
7538it [00:00, 7800.13it/s]


{'group_auc': 0.6448, 'mean_mrr': 0.2976, 'ndcg@5': 0.326, 'ndcg@10': 0.3912}
Wall time: 4min 17s


In [ ]:
store_metadata("group_auc", res_syn['group_auc'])
store_metadata("mean_mrr", res_syn['mean_mrr'])
store_metadata("ndcg@5", res_syn['ndcg@5'])
store_metadata("ndcg@10", res_syn['ndcg@10'])

In [ ]:
os.makedirs(model_path, exist_ok=True)
model.model.save_weights(os.path.join(model_path, "naml_ckpt"))